In [1]:
import numpy as np
from scipy.signal import find_peaks
from Signal_generator.generate_signal import generate_steering_vector, generate_R_hat, generate_X_matrix
import itertools

def generate_training_data(J, T, M, Q, K, SNRTRAIN, phi_max, rho):
    X = generate_X_matrix(J, T, M, SNRTRAIN)
    angles = music_method(X, J, K, phi_max, rho)
    return X, angles

def music_method(X, J, K, phi_max, rho):
    angles_combinations = generate_combinations(phi_max, rho, K)
    music_spectrum = np.zeros_like(angles_combinations, dtype=float)

    for idx, angles in enumerate(angles_combinations):
        steering_vectors = [generate_steering_vector(J, angle) for angle in angles]
        R_hat = generate_R_hat(X, steering_vectors)
        _, eigenvectors = np.linalg.eigh(R_hat)
        noise_subspace = eigenvectors[:, :J - K]
        
        for theta in range(len(music_spectrum)):
            steering_vector = generate_steering_vector(J, theta)
            music_spectrum[idx, theta] = 1 / np.linalg.norm(noise_subspace.conj().T @ steering_vector)
    
    estimated_angles = estimate_angles(music_spectrum)
    return estimated_angles

def generate_combinations(phi_max, rho, nb_sources):
    """Cette fonction sert à générer toutes les combinaisons d'angles possibles dans la plage de résolution"""
    # Générer une plage de valeurs possibles pour les signaux
    values = list(range(-int(phi_max / rho), int(phi_max / rho) + 1))
    
    # Générer toutes les combinaisons possibles de signaux
    all_combinations = list(itertools.product(values, repeat=nb_sources))
    
    # Supprimer les combinaisons où l'ordre ne compte pas
    unique_combinations = set()
    for combination in all_combinations:
        sorted_combination = tuple(sorted(combination))
        unique_combinations.add(sorted_combination)
    unique_combinations=list(unique_combinations)
        
    #supprimer les combinaisons ou deux signaux ont la même valeur
    combination_a_supp=[]
    for combination in unique_combinations:
        presents=[]
        for element in combination:
            if element not in presents:
                presents.append(element)
            else:
                combination_a_supp.append(combination)
    for combination in combination_a_supp:
        unique_combinations.remove(combination)
                
    return unique_combinations

def estimate_angles(music_spectrum):
    estimated_angles = []
    for spectrum in music_spectrum:
        all_peaks, _ = find_peaks(np.real(spectrum), height=0)  # height=0 pour inclure tous les pics
        sorted_peaks = sorted(all_peaks, key=lambda x: spectrum[x], reverse=True)  # Triez les pics par amplitude dans l'ordre décroissant
        top_peaks = sorted(sorted_peaks[:1])  # Sélectionnez le plus grand pic
        estimated_angles.append(top_peaks[0])  # Obtenez l'angle estimé correspondant au plus grand pic
    return estimated_angles